<a href="https://colab.research.google.com/github/Sujana-Mollick/Website/blob/main/Clickstream.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Reading CSV FIle**

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
Data = pd.read_csv('/content/drive/My Drive/Clickstream/Data2.csv')

In [ ]:
Data

In [ ]:
print(Data.isnull().sum())

In [ ]:
for column in Data.columns:
    print(f"Total Uniqe Value Of: {column}: {len(Data[column].unique())} ")
    print('___________________________________________')

In [ ]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

In [ ]:
def convert_to_list(sequence):
    return sequence.split(',')

In [ ]:
Data['Sequence'] = Data['Sequence'].apply(convert_to_list)

Data['SequenceDurations'] = Data['SequenceDurations'].apply(lambda x: list(map(int, x.split(','))))

Data['AverageDuration'] = Data['SequenceDurations'].apply(lambda x: sum(map(int, x)) / len(x) if len(x) > 0 else 0)

te = TransactionEncoder()
one_hot_encoded = te.fit_transform(Data['Sequence'])
df = pd.DataFrame(one_hot_encoded, columns=te.columns_)


In [ ]:
Data

In [ ]:
df

In [ ]:
import matplotlib.pyplot as plt
sequences = Data['Sequence']
flat_sequence = [item for sublist in sequences for item in sublist]
value_counts = pd.Series(flat_sequence).value_counts()

plt.figure(figsize=(14, 7))
plt.bar(value_counts.index, value_counts.values)
plt.xlabel('Visited Page')
plt.ylabel('Number of Visit:')
plt.title('Analysis Page Visit')
plt.show()

In [ ]:
from sklearn import preprocessing
Data['session_id']=preprocessing.LabelEncoder().fit_transform(Data['session_id'])
Data['AverageDuration']=Data['AverageDuration'].apply(int)
Data

In [ ]:
Extract=Data.drop(['SequenceDurations','Sequence'],axis=1)
Extract

In [ ]:
import seaborn as sns
sns.relplot(x="session_id", y="AverageDuration", kind="line", data=Extract ,height=4, aspect=15/6)

In [ ]:
df

In [ ]:
#Fp growth
from mlxtend.frequent_patterns import fpgrowth
FPG_frequent_sequences = fpgrowth(df, min_support=0.2, use_colnames=True)

# Apriori algorithm
AP_frequent_sequences = apriori(df, min_support=.2, use_colnames=True)

In [ ]:
print("Apriori Frequent Sequences:")
print(AP_frequent_sequences)
print("===========================================")
print("FP Growth Frequent Sequences:")
print(FPG_frequent_sequences)


print("\nAverage Duration for Each Session:")
print(Data[['session_id', 'AverageDuration']])

In [ ]:
from mlxtend.frequent_patterns import association_rules

In [ ]:
print('Apriori:')
rules = association_rules(AP_frequent_sequences, metric ="lift", min_threshold = 1)
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])
rules

In [ ]:
print('FP Growth:')
rules = association_rules(FPG_frequent_sequences, metric ="lift", min_threshold = 1)
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])
rules